<a href="https://colab.research.google.com/github/maoroh/keras-frcnn/blob/master/CountingCars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget 

--2019-02-02 16:54:00--  https://drive.google.com/uc?export=download
Resolving drive.google.com (drive.google.com)... 74.125.70.138, 74.125.70.101, 74.125.70.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.70.138|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2019-02-02 16:54:00 ERROR 400: Bad Request.



In [0]:

!git clone https://github.com/maoroh/keras-frcnn.git

%cd /content/keras-frcnn
%mkdir model

In [0]:
!wget http://139.59.179.174/cars_last_model.hdf5 -O model/cars_last_model.hdf5
!wget http://139.59.179.174/resnet50_weights_tf_dim_ordering_tf_kernels.h5 -O model/resnet50_weights_tf_dim_ordering_tf_kernels.h5

**Install Keras 2.0.3 instead of 2.2.4 **

In [0]:
!pip install keras==2.0.3

**Generate the annotations file from CARPK dataset**

In [0]:
import os
import glob
class AnnotationFileGenerator:
    def __init__(self,path_to_annotations_folder, images_path,output_file):
        self.path_to_annotations_folder = path_to_annotations_folder
        self.images_path = images_path
        self.output_file = output_file

    def generate(self):
        with open(self.output_file, 'r') as train_file:
            train_file_content = train_file.readlines()
            train_files_list = [x.strip() for x in train_file_content] 
            print(train_files_list)
          
        files_list = glob.glob(self.path_to_annotations_folder + '/*.txt')
        with open('./data/' + "train.txt", 'w+') as converted_file:
            for f in files_list:
                with open(f,'r') as original_file:
                    file_name = os.path.basename(f).split('.')[0]
                    if file_name in train_files_list:
                       
                        for line in original_file:
                            #split the line according spaces
                            converted_file.write(self.images_path + file_name + '.png,')
                            converted_file.write(self.convert_line(line))
                            converted_file.write('\n')
                            
                            
                        

    def convert_line(self,line):
        line_split = line.strip().split(' ')
        #taking all details for the bounding box
        (x1, y1, x2, y2, class_name) = line_split
        line_print = x1+','+y1+','+x2+','+y2+','+'Car'
        return line_print + ' '

parser = AnnotationFileGenerator(path_to_annotations_folder = './CARPK_devkit/data/Annotations',images_path = './datasets/CARPK_devkit/data/Images/',output_file = './data/train.txt')
parser.generate()

**Create a dictionary for dataset images and then use it in test phase **

In [0]:

import glob
import os
class ImagesDict:
    def __init__(self, path_to_dataset_folder):
        self.path_to_dataset_folder = path_to_dataset_folder
        self.parser_dict = {}
    
    def parseAll(self):
        self.parseAnnotations()
        self.parseImageSets()
        return self.parser_dict

    #Parsing the image set for each image
    def parseImageSets(self):
        #Read train.txt file
        with open(self.path_to_dataset_folder + '/ImageSets/train.txt','r') as train_file:
            t = [line[:-1] for line in train_file]
            for train_file_name in t:
                self.parser_dict[train_file_name]['imageset'] = 'train'
         #Read test.txt file       
        with open(self.path_to_dataset_folder + '/ImageSets/test.txt','r') as test_file:
            t = [line[:-1] for line in test_file]
            for train_file_name in t:
                self.parser_dict[train_file_name]['imageset'] = 'test'
       
    def parseAnnotations(self):
        #Getting all file paths
        files_list = glob.glob(self.path_to_dataset_folder + '/Annotations/*.txt')
       
        #Loop over all files
        for f in files_list :
            #Getting file name
            file_name = os.path.splitext(os.path.basename(f))[0]
    
            #init dictionary for this file
            self.parser_dict[file_name] = {}
            #open file in read mode
            with open(f,'r') as file:
                #count lines
                data = file.read()
                self.parser_dict[file_name]['count']  = len(data.split('\n')) - 1
                
                

basicParser = ImagesDict(path_to_dataset_folder='./datasets/CARPK_devkit/data')
dict = basicParser.parseAll()
print('Dictionary creation completed!')

   

{'count': 4, 'imageset': 'train'}


In [0]:
!python train.py


[Errno 2] No such file or directory: '/content/gdrive/My Drive/keras_frcnn'
/content/keras-frcnn
Using TensorFlow backend.
Parsing annotation files
Traceback (most recent call last):
  File "train.py", line 256, in <module>
    train_kitti()
  File "train.py", line 38, in train_kitti
    all_images, classes_count, class_mapping = get_data(cfg.simple_label_file)
  File "/content/keras-frcnn/keras_frcnn/simple_parser.py", line 41, in get_data
    (rows, cols) = img.shape[:2]
AttributeError: 'NoneType' object has no attribute 'shape'


In [0]:
%cd /content/gdrive/'My Drive'/keras_frcnn
#!python main.py


/content/gdrive/My Drive/keras_frcnn


**Test Phase - Calc MAE And RMSE** 

In [0]:
from detector import FasterRCNNDetector
import cv2
import math

def main():
    mae = 0.0
    rmse = 0.0
    #Create Detector object
    detector = FasterRCNNDetector(model_path='../cars_last_model.hdf5')
    count = 0
    
    for key, value in dict.items():
      print(key)
      if(value['imageset'] == 'test'):
          count = count + 1
          test_predict_count = value['count']
          #Read an Image from the dataset
          img = cv2.imread('./datasets/CARPK_devkit/data/Images/' + key + '.png') 
          predicted_boxes = detector.detect_on_image(img)
          actual_predict_count = len(predicted_boxes[0])
          error = abs(test_predict_count - actual_predict_count)
          mae = mae + error
          rmse = rmse + pow(error,2) 
          print(actual_predict_count)
          print(test_predict_count)
    
    mae = mae / count
    rmse = math.sqrt((rmse / count))
    print("MAE is : ")
    print (mae)
    print("RMSE is : ")
    print (rmse)


if __name__ == '__main__':
    main()

In [0]:


from detector import FasterRCNNDetector
import cv2


def main():
    mae = 0.0
    detector = FasterRCNNDetector(model_path='./model/cars_last_model.hdf5')
    img = cv2.imread('./datasets/CARPK_devkit/data/Images/20161030_GF2_00146.png') 
    detector.detect_on_image(img)
         


if __name__ == '__main__':
    main()

NOTE: this code only support to keras 2.0.3, newest version this line will got errors. see trace back.
Loading weights from ./model/cars_last_model.hdf5
Car:
[556. 115. 595. 191.] prob: 0.9998986721038818
[ 979.  383. 1017.  479.] prob: 0.9997758269309998
[  0. 364.  57. 460.] prob: 0.9997631907463074
[364. 383. 403. 479.] prob: 0.9997407793998718
[1113.  633. 1171.  710.] prob: 0.9996166229248047
[710. 115. 748. 211.] prob: 0.9996045231819153
[518. 383. 556. 460.] prob: 0.9996015429496765
[518. 115. 556. 211.] prob: 0.9995449185371399
[364. 633. 403. 710.] prob: 0.9994833469390869
[326. 633. 364. 710.] prob: 0.9994714856147766
[844. 383. 883. 479.] prob: 0.9994235038757324
[748. 115. 787. 211.] prob: 0.999276340007782
[940. 134. 979. 230.] prob: 0.9991568326950073
[ 979.  134. 1017.  211.] prob: 0.9990673661231995
[1151.  115. 1209.  230.] prob: 0.998900294303894
[422. 633. 460. 710.] prob: 0.9988663196563721
[556. 383. 595. 479.] prob: 0.9987528324127197
[230. 383. 268. 460.] prob: 0